In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from firthlogist import FirthLogisticRegression
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from tqdm import tqdm

In [ ]:
def generate_sample(
    n_sample: int, beta: np.ndarray, rng: np.random.Generator
    ) -> tuple[np.ndarray, np.ndarray]:
    n_feature = len(beta)
    x = rng.normal(size=(n_sample, n_feature))
    y = rng.binomial(1, 1 / (1 + np.exp(-x @ beta)))
    return x, y

models = [  # (method_name, model_constructor, model_kwargs)
    ("Firth", FirthLogisticRegression, {"fit_intercept": False, "max_iter": 1000}),
    ("normal", LogisticRegression, {
        "fit_intercept": False, "max_iter": 1000, "penalty": None
        }
     ),
    ("CV_L2", LogisticRegressionCV, {
        "fit_intercept": False, "max_iter": 1000, "penalty": "l2",
        "cv": 5, "Cs": [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
        }),
    ("L2", LogisticRegression, {
        "fit_intercept": False, "max_iter": 1000, "penalty": "l2", "C": 10
        }
     ),
]

In [ ]:
rng = np.random.default_rng(0)
MC = 500
n_sample = 30
n_feature = 5

beta = rng.uniform(-1, 1, size=n_feature)
print(beta)

method2coefs = {method_name: np.empty((MC, n_feature)) for method_name, _, _ in models}

for i in tqdm(range(MC)):
    for method_name, model_constructor, model_kwargs in models:
        X, y = generate_sample(n_sample, beta, rng)
        model = model_constructor(**model_kwargs)
        model.fit(X, y)
        method2coefs[method_name][i, :] = model.coef_.squeeze()

# plot the bias
fig, axs = plt.subplots(
    len(method2coefs), n_feature,
    figsize=(3*n_feature, 3*len(method2coefs))
    )
for i, method in enumerate(method2coefs.keys()):
    bias = method2coefs[method] - beta
    for j in range(n_feature):
        sns.boxplot(bias[:, j], ax=axs[i, j])
        axs[i, j].set_title(
            f'{method}: {j+1}, \
            \n mean: {bias[:, j].mean():.2f}, std: {bias[:, j].std():.2f}'
            )
        axs[i, j].axhline(0, color='r', linestyle='--')
fig.suptitle(
    f'Bias of Firth and Normal Logistic Regression \n \
    (n_sample={n_sample}, n_feature={n_feature}), \n \
    beta={np.round(beta, 3)}'
    )
plt.tight_layout()